In [5]:
import pandas as pd
import s3fs

df = pd.read_csv('s3://noaa-isd-pds/isd-history.csv')
df.head()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,NaN,NaN,NaN,0.00,0.000,7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,NaN,NaN,0.00,0.000,7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,NaN,NaN,0.00,0.000,7070.0,20140923,20150926
3,008260,99999,WXPOD8270,NaN,NaN,NaN,0.00,0.000,0.0,20050101,20100920
4,008268,99999,WXPOD8278,AF,NaN,NaN,32.95,65.567,1156.7,20100519,20120323


In [6]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:2.7.3" pyspark-shell'
from pyspark import SparkConf
from pyspark import SparkContext
conf = SparkConf()\
    .setMaster("local")\
    .setAppName("pyspark-unittests")\
    .set("spark.sql.parquet.compression.codec", "snappy")
sc = SparkContext(conf = conf)

sc._jsc.hadoopConfiguration().set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set('fs.s3a.access.key', 'insert your own access key')
sc._jsc.hadoopConfiguration().set('fs.s3a.secret.key', 'insert your own secret')
inputFile = sc.textFile("s3://noaa-isd-pds/isd-history.csv")

NameError: name 'sparkContext' is not defined

In [44]:
inputFile = sc.textFile("s3://noaa-isd-pds/data/2020/297260-99999-2020.gz,s3://noaa-isd-pds/data/2019/297260-99999-2019.gz")

In [34]:
from pyspark.sql.types import StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DateType, FloatType

schema = StructType([
    StructField("var_data_len", StringType(), True),
    StructField("usaf_station_id", StringType(), True),
    StructField("wban_station_id", StringType(), True),
    StructField("date", DateType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True),
    StructField("report_type", StringType(), True),
    StructField("elevation_meters", IntegerType(), True),
    StructField("call_letters", StringType(), True),
    StructField("qc_process", StringType(), True),
    StructField("air_temp_c", FloatType(), True),
])

In [13]:
import datetime

def value_or_missing(val):
    '''
    ISD values uses a string of 9s to indicate missing data.
    This method checks if the data is missing and returns None if it is,
    or the actual string if not
    '''
    nval = len(val)
    all_9s = "9"*nval
    signed_9s = "+" + "9" * (nval-1)
    missing = (val == all_9s) or (val == signed_9s)
    return None if missing else val

def parse_isd_line(line):
    '''
    Parses a NOAA isd line. Returns data as a dictionary.
    '''
    # Station data
    var_data_len = int(line[0:4])
    usaf_station_id = line[4:10]
    wban_station_id = line[10:15]
    date = value_or_missing(line[15:27])
    date = datetime.datetime.strptime(date, "%Y%m%d%H%M") if date is not None else None
    latitude = value_or_missing(line[28:34])
    latitude = float(latitude) / 1000.0 if latitude else None
    longitude = value_or_missing(line[34:41])
    longitude = float(longitude) / 1000.0 if longitude else None
    report_type = value_or_missing(line[41:46])
    elevation = value_or_missing(line[46:51])
    elevation = int(elevation) if elevation else None
    call_letters = value_or_missing(line[51:56])
    qc_process = value_or_missing(line[56:59])
    
    # Air temp
    air_temp_c = value_or_missing(line[87:92])
    air_temp_c = float(air_temp_c) / 10. if air_temp_c else None
    
    return {
        "var_data_len": var_data_len,
        "usaf_station_id": usaf_station_id,
        "wban_station_id": wban_station_id,
        "date": date,
        "latitude": latitude,
        "longitude": longitude,
        "report_type": report_type,
        "elevation_meters": elevation,
        "call_letters": call_letters,
        "qc_process": qc_process,
        "air_temp_c": air_temp_c,
    }

In [45]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession(sc)
from pyspark.sql import Row
from collections import OrderedDict

df = inputFile.map(lambda x: parse_isd_line(x)).toDF(schema)

In [46]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window



In [47]:
df = df.withColumn("date_trunc", F.to_date(F.col("date")))

In [48]:
agg_df = df.groupBy("date_trunc").agg({"air_temp_c": "avg"})

In [49]:
df.head()

Row(var_data_len='256', usaf_station_id='297260', wban_station_id='99999', date=datetime.date(2020, 1, 1), latitude=54.367000579833984, longitude=81.88300323486328, report_type='FM-12', elevation_meters=132, call_letters=None, qc_process='V02', air_temp_c=-13.5, date_trunc=datetime.date(2020, 1, 1))

In [50]:
agg_df.head(10)

[Row(date_trunc=datetime.date(2019, 5, 8), avg(air_temp_c)=19.475000083446503),
 Row(date_trunc=datetime.date(2019, 6, 4), avg(air_temp_c)=14.212499856948853),
 Row(date_trunc=datetime.date(2020, 1, 21), avg(air_temp_c)=-24.175000190734863),
 Row(date_trunc=datetime.date(2019, 9, 22), avg(air_temp_c)=17.05000002682209),
 Row(date_trunc=datetime.date(2019, 11, 1), avg(air_temp_c)=6.500000059604645),
 Row(date_trunc=datetime.date(2019, 11, 18), avg(air_temp_c)=-4.137499928474426),
 Row(date_trunc=datetime.date(2019, 11, 21), avg(air_temp_c)=-22.65000033378601),
 Row(date_trunc=datetime.date(2019, 5, 27), avg(air_temp_c)=13.23749989271164),
 Row(date_trunc=datetime.date(2019, 2, 23), avg(air_temp_c)=-8.937500089406967),
 Row(date_trunc=datetime.date(2020, 2, 4), avg(air_temp_c)=-5.642857074737549)]